# Collect data from the BigQuery database

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

current_path = sys.path[0]
sys.path.append(current_path[:current_path.find('defi-measurement')] + "liquidity-distribution-history")

In [3]:
import os
from tqdm.notebook import tqdm

import json

import pandas as pd

import psycopg2.extras
import pandas as pd


from dotenv import load_dotenv


load_dotenv(override=True)

True

In [4]:
postgres_uri_us = os.environ["POSTGRESQL_URI_US"]

assert postgres_uri_us is not None, "Connection string to Postgres is not set"

## Get data for the 10 biggest pools by TVL

## New approach---Just import the whole database

In [5]:
def seed_db(
    bq_table: str,
    pg_table: str,
    start_index: int,
    max_results: int,
    total_rows: int,
    where: str = "TRUE",
) -> None:
    # BigQuery client
    proj_id = "mimetic-design-338620"
    bq_dataset = 'uniswap'

    # Postgres connection
    conn = psycopg2.connect(postgres_uri_us)
    cur = conn.cursor()
    
    it = tqdm(total=total_rows, initial=start_index)

    while True:
        it.set_description(f"Inserting row {start_index:_} - {start_index + max_results:_}")
        rows = pd.read_gbq(
            f"""
                SELECT *
                FROM `{bq_dataset}.{bq_table}`
                WHERE {where}
                LIMIT {max_results}
                OFFSET {start_index}
            """,
            project_id=proj_id,
            dialect='standard',
            # progress_bar_type='tqdm'
        ).to_dict('records')

        if not rows:
            break

        # Insert the rows into Postgres only if row is not already present
        psycopg2.extras.execute_values(
            cur,
            f"""
            INSERT INTO {pg_table} VALUES %s
            ON CONFLICT DO NOTHING
            """,
            # ON CONFLICT (block_number, transaction_index, log_index) DO NOTHING
            [tuple(x.values()) for x in rows],
            template=None,
            page_size=100
        )
        conn.commit()

        # Update the start index for the next batch of rows
        start_index += max_results
        it.update(max_results)

    # Close the Postgres connection
    cur.close()
    conn.close()

In [10]:
# Define your BigQuery table and Postgres table
bq_table = 'swap'
pg_table = 'swaps'

# Fetch the data from BigQuery in chunks
start_index = 33_647_421
max_results = 1_000_000  # adjust this value based on your system's memory
total_rows = 34_986_784

seed_db(bq_table, pg_table, start_index, max_results, total_rows)

 96%|#########6| 33647421/34986784 [00:00<?, ?it/s]

In [11]:
# Define your BigQuery table and Postgres table
bq_table = 'MintBurnV3-labeled'
pg_table = 'mb'

# Fetch the data from BigQuery in chunks
start_index = 1_356_519
max_results = 100_000  # adjust this value based on your system's memory
total_rows = 1_444_617

seed_db(bq_table, pg_table, start_index, max_results, total_rows)

 94%|#########3| 1356519/1444617 [00:00<?, ?it/s]

In [12]:
# Define your BigQuery table and Postgres table
bq_table = 'V3Factory_PoolCreated'
pg_table = 'factory'

# Fetch the data from BigQuery in chunks
start_index = 13_397
max_results = 1_000  # adjust this value based on your system's memory
total_rows = 14_341

seed_db(bq_table, pg_table, start_index, max_results, total_rows)

 93%|#########3| 13397/14341 [00:00<?, ?it/s]

In [13]:
# Define your BigQuery table and Postgres table
bq_table = 'ethereum_uniswap_v3_pool_evt_initialize'
pg_table = 'initialize'

# Fetch the data from BigQuery in chunks
start_index = 13_360
max_results = 1_000  # adjust this value based on your system's memory
total_rows = 14_301

seed_db(bq_table, pg_table, start_index, max_results, total_rows)

 93%|#########3| 13360/14301 [00:00<?, ?it/s]

In [7]:
# Define your BigQuery table and Postgres table
bq_table = 'pools_tokens_decimals'
pg_table = 'token_info'

# Fetch the data from BigQuery in chunks
start_index = 0
max_results = 1_000  # adjust this value based on your system's memory
total_rows = 15_077

seed_db(bq_table, pg_table, start_index, max_results, total_rows)

  0%|          | 0/15077 [00:00<?, ?it/s]

## Remove the duplicate rows in the tables

In [8]:
import psycopg2

def remove_duplicates(pg_table, columns):
    # establish a connection
    conn = psycopg2.connect(postgres_uri_us)
    conn.autocommit = False  # start a new transaction

    # create a cursor
    cur = conn.cursor()

    batch_size = 100_000  # number of rows to delete in each batch
    row_count = batch_size  # initial value to enter the loop

    col_str = ", ".join(columns)  # columns as a string

    # create the index if it doesn't exist
    cur.execute(f"""
        CREATE INDEX IF NOT EXISTS idx_swaps_columns ON {pg_table} ({col_str});
    """)
    conn.commit()  # commit the index creation

    # loop until there's no more duplicates
    while row_count == batch_size:
        # find the duplicates
        cur.execute(f"""
            DELETE FROM {pg_table}
            WHERE ctid IN (
                SELECT ctid
                FROM (
                    SELECT ctid,
                        ROW_NUMBER() OVER(PARTITION BY {col_str} ORDER BY ctid) AS rn
                    FROM {pg_table}
                ) t
                WHERE t.rn > 1
                LIMIT %s
            )
        """, (batch_size,))
        
        row_count = cur.rowcount  # get the number of deleted rows

        # commit the deletion
        conn.commit()

        # print the progress
        print(f"Deleted {row_count:_} rows in this iteration")

    # close the cursor and the connection
    cur.close()
    conn.close()


remove_duplicates('swaps', ['block_number', 'transaction_index', 'log_index'])

In [9]:
remove_duplicates('mb', ['block_number', 'transaction_index', 'log_index'])

Deleted 100_000 rows in this iteration
Deleted 100_000 rows in this iteration
Deleted 100_000 rows in this iteration
Deleted 100_000 rows in this iteration
Deleted 100_000 rows in this iteration
Deleted 100_000 rows in this iteration
Deleted 100_000 rows in this iteration
Deleted 100_000 rows in this iteration
Deleted 100_000 rows in this iteration
Deleted 23_317 rows in this iteration


In [11]:
remove_duplicates('factory', ['pool'])

Deleted 13_397 rows in this iteration


## Work on the token info

In [21]:
proj_id = "mimetic-design-338620"

pool_info_df = pd.read_gbq(
    f"""
        SELECT *
        FROM `mimetic-design-338620.uniswap.pools_tokens_decimals`
    """,
    project_id=proj_id,
    dialect='standard',
    progress_bar_type='tqdm'
)

Downloading: 100%|██████████|


In [22]:
pool_info_df.shape

(13397, 13)

In [28]:
token0na = pool_info_df[pool_info_df.token0symbol.isna()].token0.unique()
token1na = pool_info_df[pool_info_df.token1symbol.isna()].token1.unique()

token_na_set = set(token0na).union(set(token1na))

len(token_na_set)

7897

In [44]:
# Merge token0 and token1 symbols into one column despite them having different names
df0 = pool_info_df[['token0', 'token0symbol']].rename(columns={'token0': 'token', 'token0symbol': 'symbol'})
df1 = pool_info_df[['token1', 'token1symbol']].rename(columns={'token1': 'token', 'token1symbol': 'symbol'})

all_tokens = pd.concat([df0, df1]).reset_index(drop=True)

all_tokens

,token,symbol
0,0x0b5afdc93a86062a49063ea96ae944d7f966b6ae,None
1,0x111111517e4929d3dcbdfa7cce55d30d4b6bc4d6,None
2,0x0d86eb9f43c57f6ff3bc9e23d8f9d82503f0e84b,None
3,0x028171bca77440897b824ca71d1c56cac55b68a3,aDAI
4,0x1abaea1f7c830bd89acc67ec4af516284b1bc33c,None
...,...,...
26789,0xffd9802610c726c1917fc3ed8c8fbe9ea3e10583,None
26790,0xffe1ab085332720f63cf6784b94b43f45ee24c8f,DXB
26791,0xfffc918b2da0927e0c9d8595c819cbd89b1e644b,None
26792,0xfffdaa42568c2032616ba7061c690e9dccf6bdea,VNV


In [47]:
missing = all_tokens[all_tokens.symbol.isna()].token.unique()

missing.shape

(7897,)

In [68]:
missing_ordered = all_tokens.fillna("NA")[all_tokens.token.isin(missing)].groupby('token').count().sort_values('symbol', ascending=False).index
missing_ordered

Index(['0x111111517e4929d3dcbdfa7cce55d30d4b6bc4d6',
       '0x5f98805a4e8be255a32880fdec7f6728c6568ba0',
       '0xd2877702675e6ceb975b4a1dff9fb7baf4c91ea9',
       '0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0',
       '0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5',
       '0x6982508145454ce325ddbe47a25d4ec3d2311933',
       '0x52e7b23e1faba6d83376725e2ccad75efbd8736e',
       '0x3819f64f282bf135d62168c1e513280daf905e06',
       '0x0ab87046fbb341d058f17cbc4c1133f25a20a52f',
       '0x3b94440c8c4f69d5c9f47bab9c5a93064df460f5',
       ...
       '0x61c6a8d848d1477bdf58293de0fbf3b28cad5896',
       '0x61aeea25b426494788765324aceace00841bf838',
       '0x61a85b14c0ca2c19c2017b12e596e655194d4b43',
       '0x619f125bbca30c4baff5003779c47e3a4243a85c',
       '0x619abba3218226040bb97a9e4cd6f2ad84a33e6e',
       '0x6192c0e86de10cbecdaf507f1ff12df47ba60214',
       '0x6192483a0d98e4607f5147e1d0c082d152030485',
       '0x616d9c78cc0781b838ca66fe2d6d86d090211908',
       '0x615d8e5e1344b36a95f6ecd8e

In [69]:
# Perform a graphQL query to get the token symbols and decimals
import requests

query = """
{
  token(id: $id) {
    id
    symbol
    decimals
  }
}
"""

url = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3"

token_info = {}

for token in tqdm(missing_ordered):
    r = requests.post(url, json={'query': query.replace("$id", f'"{token}"')})
    token_info[token] = r.json()['data']['token']

token_info

  0%|          | 0/7897 [00:00<?, ?it/s]

{'0x111111517e4929d3dcbdfa7cce55d30d4b6bc4d6': {'id': '0x111111517e4929d3dcbdfa7cce55d30d4b6bc4d6',
  'symbol': 'ICHI',
  'decimals': '18'},
 '0x5f98805a4e8be255a32880fdec7f6728c6568ba0': {'id': '0x5f98805a4e8be255a32880fdec7f6728c6568ba0',
  'symbol': 'LUSD',
  'decimals': '18'},
 '0xd2877702675e6ceb975b4a1dff9fb7baf4c91ea9': {'id': '0xd2877702675e6ceb975b4a1dff9fb7baf4c91ea9',
  'symbol': 'LUNA',
  'decimals': '18'},
 '0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0': {'id': '0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0',
  'symbol': 'MATIC',
  'decimals': '18'},
 '0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5': {'id': '0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5',
  'symbol': 'OHM',
  'decimals': '9'},
 '0x6982508145454ce325ddbe47a25d4ec3d2311933': {'id': '0x6982508145454ce325ddbe47a25d4ec3d2311933',
  'symbol': 'PEPE',
  'decimals': '18'},
 '0x52e7b23e1faba6d83376725e2ccad75efbd8736e': {'id': '0x52e7b23e1faba6d83376725e2ccad75efbd8736e',
  'symbol': 'BURN',
  'decimals': '18'},
 '0x3819f64f28

In [70]:
len(token_info)

7897

In [82]:
from sqlalchemy import create_engine, MetaData, Table, select, update
from sqlalchemy.sql import or_

# Connect to the database
engine = create_engine(postgres_uri_us)
connection = engine.connect()

# Reflect table
metadata = MetaData()
table_name = "token_info"  # replace with your table name
table = Table(table_name, metadata, autoload_with=engine)


# Select rows where token0symbol or decimals0 are missing (assuming missing values are None)
missing_data = select(table.c).where(
    or_(table.c.token0symbol == None, table.c.decimals0 == None)
)

ResultProxy = connection.execute(missing_data)

# Fetch all rows
ResultSet = ResultProxy.fetchall()

print(f"Got {len(ResultSet)} rows with missing data")

# # Now, let's update the rows
for result in tqdm(ResultSet):
    stmt = (
        update(table)
        .where(table.c.pool == result.pool)
        .values(
            token0symbol=token_info[result.token0]["symbol"],
            decimals0=token_info[result.token0]["decimals"],
        )
    )
    connection.execute(stmt)
    connection.commit()

# Always remember to close the connection
connection.close()

Got 8121 rows with missing data


  0%|          | 0/8121 [00:00<?, ?it/s]

In [83]:
# Connect to the database
engine = create_engine(postgres_uri_us)
connection = engine.connect()

# Reflect table
metadata = MetaData()
table_name = "token_info"  # replace with your table name
table = Table(table_name, metadata, autoload_with=engine)


# Select rows where token1symbol or decimals1 are missing (assuming missing values are None)
missing_data = select(table.c).where(
    or_(table.c.token1symbol == None, table.c.decimals1 == None)
)

ResultProxy = connection.execute(missing_data)

# Fetch all rows
ResultSet = ResultProxy.fetchall()

print(f"Got {len(ResultSet)} rows with missing data")

# # Now, let's update the rows
for result in tqdm(ResultSet):
    stmt = (
        update(table)
        .where(table.c.pool == result.pool)
        .values(
            token1symbol=token_info[result.token1]["symbol"],
            decimals1=token_info[result.token1]["decimals"],
        )
    )
    connection.execute(stmt)
    connection.commit()

# Always remember to close the connection
connection.close()

Got 3509 rows with missing data


  0%|          | 0/3509 [00:00<?, ?it/s]

## Get the gas data from Big Query

In [6]:
df2 = pd.read_gbq(
    """
    select 
        swaps.block_timestamp,
        swaps.address,
        swaps.transaction_hash,
        t.nonce,
        t.gas,
        t.gas_price,
        t.receipt_cumulative_gas_used,
        t.receipt_gas_used,
        t.max_fee_per_gas,
        t.max_priority_fee_per_gas,
        t.transaction_type,
        t.receipt_effective_gas_price,
        from (
            select
            y.hash,
            block_number,
            nonce,
            gas,
            gas_price,
            receipt_cumulative_gas_used,
            receipt_gas_used,
            max_fee_per_gas,
            max_priority_fee_per_gas,
            transaction_type,
            receipt_effective_gas_price
            from `bigquery-public-data.crypto_ethereum.transactions` as y
        ) as t
        right join (
            select 
            block_number,
            block_timestamp,
            address,
            transaction_hash
            from `mimetic-design-338620.uniswap.swap` as swaps
            where swaps.address = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"
        ) as swaps
        on t.block_number = swaps.block_number
        and t.hash = swaps.transaction_hash
    """,
    project_id="mimetic-design-338620",
    dialect="standard",
    progress_bar_type="tqdm_notebook",
)

df2.shape

Downloading:   0%|          |

(4987527, 12)

In [17]:
df2 = df2.drop_duplicates()

df2.shape

(4974382, 12)

In [19]:
# Create a table in Postgres with the same schema as the DataFrame
from sqlalchemy import create_engine, MetaData, Table, Column, String, DateTime, BigInteger

# Connect to the database
engine = create_engine(postgres_uri_us)
connection = engine.connect()

# Reflect table
metadata = MetaData()
table_name = "swaps_gas"

# Use Big Integers for all numeric columns
table = Table(
    table_name,
    metadata,
    Column("block_timestamp", DateTime),
    Column("address", String),
    Column("transaction_hash", String, primary_key=True),
    Column("nonce", BigInteger),
    Column("gas", BigInteger),
    Column("gas_price", BigInteger),
    Column("receipt_cumulative_gas_used", BigInteger),
    Column("receipt_gas_used", BigInteger),
    Column("max_fee_per_gas", BigInteger),
    Column("max_priority_fee_per_gas", BigInteger),
    Column("transaction_type", BigInteger),
    Column("receipt_effective_gas_price", BigInteger),
)

metadata.create_all(engine)

# Always remember to close the connection
connection.close()

In [20]:
# Insert the data into the table
df2.to_sql(
    table_name,
    engine,
    if_exists="append",
    index=False,
    chunksize=100_000,
    method="multi",
)


4974382